Import Things needed for the project

In [3]:
%%capture
import pandas as pd
!pip install vaderSentiment
import vaderSentiment

**Loading the data**

In [23]:
cleaned = pd.read_csv("https://raw.githubusercontent.com/buildweek-saltiest-hacker/data-engineering-api/master/hacker-comments.csv")

cleaned.head()

,Unnamed: 0,hacker_name,hacker_comment,comment_saltiness,hacker_score
0,0,DanBC,Drug use has clear links with violence althoug...,9996,247.11
1,1,waps,You failed to explain why the Sana'a manuscrip...,9996,-684.35
2,2,AnthonyMouse,How do you stop another oklahoma cityI think t...,9995,507.01
3,3,michaelochurch,Can you think of any constructive ways forward...,9994,-932.92
4,4,paulhauggis,These sorts of articles do nothing to help the...,9994,-832.11


**Using Vader Sentiment Analysis**

In [13]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

score = SentimentIntensityAnalyzer()

Rober Notebook https://github.com/BrokenShell/SaltyHacker/blob/master/nlp.py

Vader Documentation

https://pypi.org/project/vaderSentiment/

Creating a the ranking for each comment.

In [26]:
# creating a new dataframe that just has the information needed

text = cleaned['hacker_comment']
name = cleaned['hacker_name']

salty_hackers = pd.DataFrame({
    'Name':name,
    'Comment': text
})

salty_hackers.head()

,Name,Comment
0,DanBC,Drug use has clear links with violence althoug...
1,waps,You failed to explain why the Sana'a manuscrip...
2,AnthonyMouse,How do you stop another oklahoma cityI think t...
3,michaelochurch,Can you think of any constructive ways forward...
4,paulhauggis,These sorts of articles do nothing to help the...


In [27]:
comment = salty_hackers['Comment']

ranking = []

for i in comment:
  scores = score.polarity_scores(i)
  final_score = scores['compound']
  rounded_score = round(final_score*10, 2)
  ranking.append(rounded_score)

In [28]:
salty_hackers['comment_ranking'] = ranking

In [29]:
salty_hackers.head()

,Name,Comment,comment_ranking
0,DanBC,Drug use has clear links with violence althoug...,-10.00
1,waps,You failed to explain why the Sana'a manuscrip...,-10.00
2,AnthonyMouse,How do you stop another oklahoma cityI think t...,-10.00
3,michaelochurch,Can you think of any constructive ways forward...,-9.99
4,paulhauggis,These sorts of articles do nothing to help the...,-9.99


In [30]:
sample_data = salty_hackers.iloc[:10]
sample_data

,Name,Comment,comment_ranking
0,DanBC,Drug use has clear links with violence althoug...,-10.00
1,waps,You failed to explain why the Sana'a manuscrip...,-10.00
2,AnthonyMouse,How do you stop another oklahoma cityI think t...,-10.00
3,michaelochurch,Can you think of any constructive ways forward...,-9.99
4,paulhauggis,These sorts of articles do nothing to help the...,-9.99
5,grey-area,Let's consider a specific example:http://www.n...,-9.99
6,danso,I don't know what it's like to be raped or sex...,-9.99
7,pdonis,"Are you actually serious? Not joking?No, I'm n...",-9.99
8,lionhearted,The person who stole it clearly needed it more...,-9.99
9,pyre,"Regarding producing oil from livestock, you'll...",-9.99


In [31]:
salty_hackers['comment_ranking'].describe()

count    230703.000000
mean          1.862038
std           5.451271
min         -10.000000
25%          -1.780000
50%           2.230000
75%           6.700000
max          10.000000
Name: comment_ranking, dtype: float64

In [32]:
average = salty_hackers.groupby(by='Name').mean()
average

,comment_ranking
Name,
001sky,1.589093
0x0,1.337746
10ren,4.157703
3pt14159,2.552662
6ren,3.444013
...,...
zdw,1.959259
zem,3.512906
zo1,2.449098


In [33]:
average[:10]

,comment_ranking
Name,
001sky,1.589093
0x0,1.337746
10ren,4.157703
3pt14159,2.552662
6ren,3.444013
Alex3917,1.733708
Alupis,2.168095
AndrewDucker,2.395503
AnimalMuppet,1.426772


In [34]:
average['comment_ranking']

Name
001sky      1.589093
0x0         1.337746
10ren       4.157703
3pt14159    2.552662
6ren        3.444013
              ...   
zdw         1.959259
zem         3.512906
zo1         2.449098
zobzu       2.499911
zokier      1.504020
Name: comment_ranking, Length: 1000, dtype: float64

In [ ]:
average_dict = average['comment_ranking'].to_dict()
all_users = average_dict.keys()
user_list = list(all_users)

In [39]:
users = salty_hackers['Name']
user_ranking = []
for user in users:
  user_rank = average_dict[user]
  round_user_rank = round(user_rank, 2)
  user_ranking.append(round_user_rank)

In [40]:
user_ranking[:10]

[-0.25, 0.68, -0.51, 0.93, 0.83, 3.94, 2.77, 1.89, 4.23, 0.58]

In [41]:
salty_hackers['user_ranking'] = user_ranking

In [42]:
salty_hackers.head()

,Name,Comment,comment_ranking,user_ranking
0,DanBC,Drug use has clear links with violence althoug...,-10.00,-0.25
1,waps,You failed to explain why the Sana'a manuscrip...,-10.00,0.68
2,AnthonyMouse,How do you stop another oklahoma cityI think t...,-10.00,-0.51
3,michaelochurch,Can you think of any constructive ways forward...,-9.99,0.93
4,paulhauggis,These sorts of articles do nothing to help the...,-9.99,0.83


In [43]:
salty_hackers['user_ranking'].describe()

count    230703.000000
mean          1.862013
std           0.943609
min          -0.800000
25%           1.190000
50%           1.810000
75%           2.440000
max           5.890000
Name: user_ranking, dtype: float64

## Exporting Final Data Set

In [44]:
compression_opts = dict(method='zip',archive_name='salty_hackers.csv')  

salty_hackers.to_csv('salty_hackers.zip', index=False, compression=compression_opts)  